In [1]:
from vamml.Model import Model,Augments
import tensorflow as tf
import tensorflow_datasets as tfds

2025-09-15 19:16:51.506667: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Building a model with default parameters, which are set to the specs used in article.

In [2]:
model = Model.BuildModel()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=10**-4),
              loss='mse',)

2025-09-15 19:16:57.214255: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-15 19:16:58.864462: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46264 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:18:00.0, compute capability: 8.6
2025-09-15 19:16:58.871946: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46595 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:c3:00.0, compute capability: 8.6


Loading prepared tensorflow dataset compiled from experimental batches. 

In [3]:
builder = tfds.builder_from_directory(builder_dir='./data/VAMML_dataset/')
train_ds,test_ds = builder.as_dataset(split = ['train[:85%]', 'train[85%:]'],
                                      shuffle_files = True,
                                      batch_size = 32,
                                      read_config = tfds.ReadConfig(shuffle_seed = 101),
                                      as_supervised = True,
                                     )

Mapping training dataset with augmentations. This is a customized function that applies not only standard augmentations (flips, rotations, zoom), but a custom augmentation specific to this dataset. This augmentation takes the dose value (first position in dense inputs) and output image intensity, then randomly scales them inversely. ie, if the light dose and pixel intensity were initially 100 and 255, respectively, after augmentation it might be 125 and 204. This augmentation is designed to make the model recognize the correlation between voxel intensity and light dose so that it can scale voxel intensity without needing a well-predicted input intensity on the part of the operator. 

In [4]:
train_ds = train_ds.map(Augments.vamml_augments,
              num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
train_ds = train_ds.repeat()
test_ds = test_ds.repeat()

In [5]:
model.fit(train_ds, 
          validation_data=test_ds,
          steps_per_epoch=20, 
          epochs=253,
          validation_steps = 1,)

Epoch 1/253


2025-09-15 19:17:08.535516: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400
2025-09-15 19:17:08.844284: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


20/20 [==============================] - 20s 280ms/step - loss: 0.2367 - val_loss: 0.2443
Epoch 2/253
20/20 [==============================] - 3s 131ms/step - loss: 0.2076 - val_loss: 0.2358
Epoch 3/253
20/20 [==============================] - 3s 131ms/step - loss: 0.1855 - val_loss: 0.2269
Epoch 4/253
20/20 [==============================] - 3s 132ms/step - loss: 0.1703 - val_loss: 0.2163
Epoch 5/253
20/20 [==============================] - 3s 132ms/step - loss: 0.1572 - val_loss: 0.2031
Epoch 6/253
20/20 [==============================] - 3s 129ms/step - loss: 0.1494 - val_loss: 0.1856
Epoch 7/253
20/20 [==============================] - 3s 134ms/step - loss: 0.1406 - val_loss: 0.1652
Epoch 8/253
20/20 [==============================] - 3s 132ms/step - loss: 0.1303 - val_loss: 0.1485
Epoch 9/253
20/20 [==============================] - 3s 131ms/step - loss: 0.1201 - val_loss: 0.1350
Epoch 10/253
20/20 [==============================] - 3s 131ms/step - loss: 0.1217 - val_loss: 0.1238


In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image (InputLayer)             [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d0_1 (Conv2D)             (None, 256, 256, 8)  80          ['image[0][0]']                  
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 8)  32         ['conv2d0_1[0][0]']              
 alization)                                                                                       
                                                                                              